## Dependencies - Setup

In [1]:
!pip install torch==2.2.2 torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121
!pip install unsloth==2025.2.15
!pip install unsloth-zoo==2025.2.7
!pip install --no-deps xformers trl peft accelerate bitsandbytes

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.3/757.3 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 111.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.1/107.1 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: unsloth-zoo
    Found existing installation: unsloth_zoo 2025.3.8
    Uninstalling unsloth_zoo-2025.3.8:
      Successfully uninstalled unsloth_zoo-2025.3.8


## Login - WANDB

In [ ]:
import wandb

wb_token = userdata.get("WANDB_API_KEY")
wandb.login(key=wb_token)

# Initialize W&B run
run = wandb.init(
    project="MedMCQA-DeepSeek-8B",
    job_type="training",
    anonymous="allow"
)

## Load Dataset

In [14]:
from datasets import load_dataset

dataset = load_dataset("PM234/MedQA-SFT", split="train[0:10000]")
print(dataset[0])

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

medqa.jsonl:   0%|          | 0.00/179M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

{'id': 'e9ad821a-c438-4965-9f77-760819dfa155', 'question': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma', 'opa': 'Hyperplasia', 'opb': 'Hyperophy', 'opc': 'Atrophy', 'opd': 'Dyplasia', 'cop': 2, 'choice_type': 'single', 'exp': 'Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950', 'subject_name': 'Anatomy', 'topic_name': 'Urinary tract', 'instruction': 'Answer the following medical question.', 'input': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma', 'output': 'Atrophy'}


In [15]:
dataset.column_names

['id',
 'question',
 'opa',
 'opb',
 'opc',
 'opd',
 'cop',
 'choice_type',
 'exp',
 'subject_name',
 'topic_name',
 'instruction',
 'input',
 'output']

## Load Model and Tokenize the Dataset

In [16]:
from unsloth import FastLanguageModel
import torch

# Load DeepSeek R1 model
max_seq_length = 2048
model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None
)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


0it [00:00, ?it/s]

🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [17]:
medical_prompt = """This is a medical question-answering task. Provide the correct answer based on the question given.

### Instruction:
{}

### Input:
{}

### Output:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = medical_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

tokenized_dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

## Fine-Tune with PEFT - SFT


> Used LoRA with Unsloth



In [19]:
from trl import SFTTrainer, SFTConfig

# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none"
)

# Train with optimized settings
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        learning_rate=1e-4,
        lr_scheduler_type="linear",
        max_steps=80,
        output_dir="./finetuned_medqa_model",
        fp16=True,
        bf16=False,
        logging_steps=5,
        report_to="wandb" # Enable W&B logging
    )
)
trainer.train()

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Converting train dataset to ChatML (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 80
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
5,3.307300
10,2.289800
15,1.518700
20,1.335300
25,1.175400
30,1.158000
35,1.130100
40,1.138500
45,1.150800
50,1.058500


TrainOutput(global_step=80, training_loss=1.3528646171092986, metrics={'train_runtime': 351.4914, 'train_samples_per_second': 1.821, 'train_steps_per_second': 0.228, 'total_flos': 1921009894883328.0, 'train_loss': 1.3528646171092986})

In [ ]:
# Finish W&B run
run.finish()

## Model Inference after Fine-Tune

In [21]:
FastLanguageModel.for_inference(model)
test_input = "### Input:\nWhat is the primary source of energy for the human body?\n### Output:"
inputs = tokenizer(test_input, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Input:
What is the primary source of energy for the human body?
### Output: Glucose


## Save Model

In [22]:
# Save model adapters locally
model.save_pretrained("finetuned_medqa")
tokenizer.save_pretrained("finetuned_medqa")

('finetuned_medqa/tokenizer_config.json',
 'finetuned_medqa/special_tokens_map.json',
 'finetuned_medqa/tokenizer.json')